In [1]:
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

# List of vectors in some dimension, with many
# more vectors than there are dimensions
num_vectors = 100
vector_len = 10
init_matrix = torch.randn(num_vectors, vector_len)
init_matrix /= init_matrix.norm(p=2, dim=1, keepdim=True)  # Normalize
init_matrix.requires_grad_(True)

def draw_orthogonality(matrix):
    n_row = matrix.shape[0]
    # Angle distribution
    dot_products = matrix @ matrix.T
    # 计算了 matrix 中每个向量的范数
    norms = torch.sqrt(torch.diag(dot_products))
    normed_dot_products = dot_products / torch.outer(norms, norms)
    # 反余弦计算 matrix 中所有向量之间的角度
    angles_degrees = torch.rad2deg(torch.acos(normed_dot_products.detach()))

    # ignore self-orthogonality.
    self_orthogonality_mask = ~(torch.eye(n_row, n_row).bool())
    self_orthogonality_mask = self_orthogonality_mask.view(n_row, n_row)  # 确保是二维的
    plt.hist(angles_degrees[self_orthogonality_mask].numpy().ravel(), bins=1000, range=(0, 180))
    plt.grid(1)
    plt.show()

draw_orthogonality(init_matrix)


ModuleNotFoundError: No module named 'torch'

In [ ]:
def optimiz(matrix, n_iter):
    
    # Set up an Adam Optimization loop to create nearly-orthogonal vectors
    optimizer = torch.optim.Adam([matrix], lr=0.01)  # 将big_matrix放入列表中
    
    n_row = matrix.shape[0]
    losses = []
    
    dot_diff_cutoff = 0.01
    
    # 一个二维的单位矩阵（identity matrix），即完美正交矩阵
    big_iden = torch.eye(n_row, n_row)
    
    for step_num in tqdm(range(n_iter)):
        optimizer.zero_grad()
    
        dot_products = matrix @ matrix.T
        
        # Punish deviation from orthogonal
        diff = dot_products - big_iden
        loss = (diff.abs() - dot_diff_cutoff).relu().sum()
    
        # Extra incentive to keep rows normalized
        loss += n_row * diff.diag().pow(2).sum()
    
        loss.backward()
        optimizer.step()
        losses.append(loss.item())

    # Loss curve
    plt.plot(losses)
    plt.grid(True)
    plt.show()

    return matrix



In [ ]:
num_steps = 100

opt_matrix = optimiz(init_matrix,  num_steps)

In [ ]:
draw_orthogonality(opt_matrix)

## Origin code

In [ ]:

# losses = []

# dot_diff_cutoff = 0.01

# # 一个二维的单位矩阵（identity matrix），即完美正交矩阵
# big_iden = torch.eye(num_vectors, num_vectors)

# for step_num in tqdm(range(num_steps)):
#     optimizer.zero_grad()

#     dot_products = big_matrix @ big_matrix.T
    
#     # Punish deviation from orthogonal
#     diff = dot_products - big_iden
#     loss = (diff.abs() - dot_diff_cutoff).relu().sum()

#     # Extra incentive to keep rows normalized
#     loss += num_vectors * diff.diag().pow(2).sum()

#     loss.backward()
#     optimizer.step()
#     losses.append(loss.item())

# # Loss curve
# plt.plot(losses)
# plt.grid(True)
# plt.show()

# # Angle distribution
# dot_products = big_matrix @ big_matrix.T
# norms = torch.sqrt(torch.diag(dot_products))
# normed_dot_products = dot_products / torch.outer(norms, norms)
# angles_degrees = torch.rad2deg(torch.acos(normed_dot_products.detach()))

# # Use this to ignore self-orthogonality.
# self_orthogonality_mask = ~(torch.eye(num_vectors, num_vectors).bool())
# self_orthogonality_mask = self_orthogonality_mask.view(num_vectors, num_vectors)  # 确保是二维的
# plt.hist(angles_degrees[self_orthogonality_mask].numpy().ravel(), bins=1000, range=(0, 180))
# plt.grid(1)
# plt.show()